In [1]:
import pandas as pd

# Carga del archivo generado por el ETL actualizado
df = pd.read_parquet("../data/historico_us.parquet")

# Primeras filas
df.head()


,state,Country_Region,confirmed,deaths,recovered,active,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,...,People_Tested,Mortality_Rate,report_date,new_cases,new_deaths,population,lat,long,incident_rate,cfr
0,Alabama,US,3667,93,0,0,74.788122,NaN,437.0,NaN,...,62029.0,2.536133,2020-04-12,0.0,0.0,4903185.0,32.3182,-86.9023,74.788122,0.0254
1,Alabama,US,3870,99,0,0,78.928288,NaN,457.0,NaN,...,64636.0,2.558140,2020-04-13,203.0,6.0,4903185.0,32.3182,-86.9023,78.928288,0.0256
2,Alabama,US,4041,114,0,0,82.415817,NaN,493.0,NaN,...,67557.0,2.821084,2020-04-14,171.0,15.0,4903185.0,32.3182,-86.9023,82.415817,0.0282
3,Alabama,US,4307,118,0,0,87.840863,NaN,525.0,NaN,...,70823.0,2.739726,2020-04-15,266.0,4.0,4903185.0,32.3182,-86.9023,87.840863,0.0274
4,Alabama,US,4465,133,0,0,91.063258,NaN,553.0,NaN,...,73878.0,2.978723,2020-04-16,158.0,15.0,4903185.0,32.3182,-86.9023,91.063258,0.0298


In [3]:
verificacion = []

# 1. Duplicados
dups = df.duplicated(subset=["state", "report_date"]).sum()
verificacion.append({
    "Chequeo": "Duplicados por estado y fecha",
    "Resultado": dups,
    "Interpretación": "✅ Sin duplicados" if dups == 0 else "⚠️ Revisar duplicados"
})

# 2. Columnas con más del 10% de nulos
nulos = df.isna().mean().sort_values(ascending=False)
for col, ratio in nulos.items():
    if ratio > 0.1:
        verificacion.append({
            "Chequeo": f"Nulos en '{col}'",
            "Resultado": f"{ratio:.2%}",
            "Interpretación": "⚠️ Revisar o justificar"
        })

pd.DataFrame(verificacion)


,Chequeo,Resultado,Interpretación
0,Duplicados por estado y fecha,0,✅ Sin duplicados
1,Nulos en 'People_Tested',80.82%,⚠️ Revisar o justificar
2,Nulos en 'Mortality_Rate',80.48%,⚠️ Revisar o justificar
3,Nulos en 'Total_Test_Results',40.54%,⚠️ Revisar o justificar
4,Nulos en 'Testing_Rate',25.47%,⚠️ Revisar o justificar
5,Nulos en 'Case_Fatality_Ratio',20.43%,⚠️ Revisar o justificar
6,Nulos en 'incident_rate',13.82%,⚠️ Revisar o justificar
7,Nulos en 'long',13.82%,⚠️ Revisar o justificar
8,Nulos en 'lat',13.82%,⚠️ Revisar o justificar
9,Nulos en 'population',13.82%,⚠️ Revisar o justificar


In [4]:
import pandas as pd

# Crear resumen de conclusiones
resumen = [
    {
        "Aspecto Evaluado": "Duplicados por estado y fecha",
        "Resultado": "0 duplicados",
        "Acción / Interpretación": "✅ Sin duplicados"
    },
    {
        "Aspecto Evaluado": "Columnas críticas (`confirmed`, `deaths`, `new_cases`, `cfr`)",
        "Resultado": "Sin valores nulos",
        "Acción / Interpretación": "✅ Listas para análisis"
    },
    {
        "Aspecto Evaluado": "People_Tested / Mortality_Rate / Testing_Rate",
        "Resultado": "Más del 80% con nulos",
        "Acción / Interpretación": "❌ No se usarán en el dashboard"
    },
    {
        "Aspecto Evaluado": "Case_Fatality_Ratio (original)",
        "Resultado": "20% con nulos",
        "Acción / Interpretación": "🟢 Reemplazada por `cfr` calculada"
    },
    {
        "Aspecto Evaluado": "incident_rate, lat, long, population",
        "Resultado": "13% con nulos",
        "Acción / Interpretación": "⚠️ Verificar `state_metadata.csv`"
    }
]

# Mostrar como DataFrame estilizado
df_resumen = pd.DataFrame(resumen)

df_resumen.style.set_properties(**{
    'background-color': 'white',
    'border': '1px solid #ddd',
    'text-align': 'left'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#f2f2f2'), ('font-weight', 'bold')]}
])


,Aspecto Evaluado,Resultado,Acción / Interpretación
0,Duplicados por estado y fecha,0 duplicados,✅ Sin duplicados
1,"Columnas críticas (`confirmed`, `deaths`, `new_cases`, `cfr`)",Sin valores nulos,✅ Listas para análisis
2,People_Tested / Mortality_Rate / Testing_Rate,Más del 80% con nulos,❌ No se usarán en el dashboard
3,Case_Fatality_Ratio (original),20% con nulos,🟢 Reemplazada por `cfr` calculada
4,"incident_rate, lat, long, population",13% con nulos,⚠️ Verificar `state_metadata.csv`


In [5]:
# Selecciona las columnas finales que usarás en el dashboard
df_final = df[["report_date", "state", "confirmed", "deaths", "new_cases", "new_deaths", "cfr"]]

# Exporta como CSV
df_final.to_csv("../data/resumen_dashboard.csv", index=False)

print("Archivo CSV exportado correctamente.")


Archivo CSV exportado correctamente.
